In [134]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from sklearn.preprocessing import LabelBinarizer
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

In [7]:
us_videos_data = pd.read_csv('USvideos.csv')
us_videos_data.describe()

,category_id,views,likes,dislikes,comment_count
count,40949.000000,4.094900e+04,4.094900e+04,4.094900e+04,4.094900e+04
mean,19.972429,2.360785e+06,7.426670e+04,3.711401e+03,8.446804e+03
std,7.568327,7.394114e+06,2.288853e+05,2.902971e+04,3.743049e+04
min,1.000000,5.490000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,17.000000,2.423290e+05,5.424000e+03,2.020000e+02,6.140000e+02
50%,24.000000,6.818610e+05,1.809100e+04,6.310000e+02,1.856000e+03
75%,25.000000,1.823157e+06,5.541700e+04,1.938000e+03,5.755000e+03
max,43.000000,2.252119e+08,5.613827e+06,1.674420e+06,1.361580e+06


In [8]:
# Splitting the data into training and test sets (80% training, 20% test)
train_data, test_data = train_test_split(us_videos_data, test_size=0.2, random_state=42)

train_data.shape, test_data.shape

((32759, 16), (8190, 16))

In [137]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize titles
train_encodings = tokenizer(train_data['title'].tolist(), truncation=True, padding=True, max_length=50)
test_encodings = tokenizer(test_data['title'].tolist(), truncation=True, padding=True, max_length=50)


loading file vocab.txt from cache at C:\Users\Keon/.cache\huggingface\hub\models--bert-base-uncased\snapshots\1dbc166cf8765166998eff31ade2eb64c8a40076\vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\Keon/.cache\huggingface\hub\models--bert-base-uncased\snapshots\1dbc166cf8765166998eff31ade2eb64c8a40076\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\Keon/.cache\huggingface\hub\models--bert-base-uncased\snapshots\1dbc166cf8765166998eff31ade2eb64c8a40076\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps

In [138]:
train_data

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
32650,PAHm7SEhhuw,18.04.05,Boxing with Evander Holyfield & Joel McHale | ...,LOL Network,23,2018-04-26T17:00:02.000Z,"What the Fit|""Kevin Hart What the Fit""|""Kevin ...",1103235,20418,1125,1238,https://i.ytimg.com/vi/PAHm7SEhhuw/default.jpg,False,False,False,If there’s one thing that boxing legend Evande...
29588,-xAMnJgc3oA,18.19.04,Janelle Monáe – I Like That [Official Audio],Janelle Monáe,10,2018-04-16T04:38:11.000Z,"Janelle Monae|""Wonderland""|""Janelle""|""Electric...",382903,16787,245,1060,https://i.ytimg.com/vi/-xAMnJgc3oA/default.jpg,False,False,False,"“I Like That”, “Make Me Feel”, “Django Jane” &..."
20521,OKXLBEvGgTA,18.26.02,I Tried Following a Safiya Nygaard Makeup Tuto...,Sylvia Gani,26,2018-02-19T01:30:53.000Z,"i tried following a makeup tutorial|""I tried f...",799025,32450,763,2393,https://i.ytimg.com/vi/OKXLBEvGgTA/default.jpg,False,False,False,SUBSCRIBE HERE: http://bit.ly/2g1cqFW\nWatch t...
2008,Wen6VQS6NG4,17.24.11,Giant Mousetrap powered Car,TheBackyardScientist,28,2017-11-23T02:47:34.000Z,"mousetrap car|""thebackyardscientist""|""Mousetra...",588908,16422,1238,1636,https://i.ytimg.com/vi/Wen6VQS6NG4/default.jpg,False,False,False,This is just a mini-video to announce i've bee...
12586,gsVgqms1slE,18.17.01,Rag'n'Bone Man - Die Easy (Official Video),RagnBoneManVEVO,10,2018-01-11T08:00:01.000Z,"Alternative/Indie|""Columbia""|""Die Easy""|""Rag'n...",299897,13180,294,626,https://i.ytimg.com/vi/gsVgqms1slE/default.jpg,False,False,False,Music video by Rag'n'Bone Man performing Die E...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,3j5eGupDWY4,17.15.12,"Homeless and abused, this Pit Bull didn't lose...",Hope For Paws - Official Rescue Channel,15,2017-12-12T17:01:47.000Z,"Loreta Frankonyte|""Eldad Hagar""|""Hope For Paws""",394927,22272,167,1366,https://i.ytimg.com/vi/3j5eGupDWY4/default.jpg,False,False,False,We need your support this holiday season! Ple...
11284,aCoDqL1dK9c,18.09.01,BEST MAKEUP OF 2017!,NikkieTutorials,26,2018-01-05T21:00:29.000Z,"best makeup of 2017|""best makeup""|""best makeup...",1528291,159801,723,78990,https://i.ytimg.com/vi/aCoDqL1dK9c/default.jpg,False,False,False,Make sure you subscribe to my channel and hit ...
38158,gS1DbvHHVH0,18.01.06,Going in to brain surgery,Simone Giertz,28,2018-05-30T14:22:13.000Z,[none],1250826,109170,825,19039,https://i.ytimg.com/vi/gS1DbvHHVH0/default.jpg,False,False,False,We’ll post an update on Instagram and Twitter ...
860,svZScJMh4Pg,17.18.11,Dua Lipa - Golden Slumbers,Dua Lipa,10,2017-11-16T17:02:13.000Z,"dua lipa|""golden slumbers""|""xmas songs""|""chris...",441164,36355,560,2482,https://i.ytimg.com/vi/svZScJMh4Pg/default.jpg,False,False,False,Production By Dan Heath and Superhuman\n\n\nMY...


In [9]:
# One-hot coding for the video ids
# One-hot encoding doesn't work since the model expects 1 integer instead of a vector(one-hot encoding) as label
lb = LabelBinarizer()
train_labels = lb.fit_transform(train_data['video_id'])
test_labels = lb.transform(test_data['video_id'])

train_labels.shape, test_labels.shape

((32759, 6171), (8190, 6171))

In [140]:
len(lb.classes_)

6171

In [141]:
# Convert video_id to class indices
train_label_indices = train_data['video_id'].map(lambda x: list(lb.classes_).index(x)).values
# test_label_indices = test_data['video_id'].map(lambda x: list(lb.classes_).index(x)).values # doesn't work because every label is unique


In [142]:
class YouTubeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = YouTubeDataset(train_encodings, train_label_indices)
# val_dataset = YouTubeDataset(test_encodings, test_labels)

In [143]:
train_dataset[0], train_dataset[1]

({'input_ids': tensor([  101,  8362,  2007,  9340,  4063,  4151,  3790,  1004,  8963, 11338,
          15238,  1064,  4901,  7530,  1024,  2054,  1996,  4906,  4958,  1022,
           1064,  4756,  2041,  5189,  2897,   102,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
  'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0]),
  'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0]),
  'labels': tensor(2549)},
 {'input_ids': tensor([  101,  4869,  6216, 13813,  2063,  1516,  1045,  2066,  2008,  1031,
           2880,  5746,  1033,   102,     0,     0,     0,     0,

In [144]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [2]:
# Training

# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(lb.classes_))

# training_args = TrainingArguments(
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=5,
#     logging_dir='./logs',
#     output_dir='./output'
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset
# )

# trainer.train()

NameError: name 'BertForSequenceClassification' is not defined

In [3]:
new_titles = ["WE WANT TO TALK ABOUT OUR MARRIAGE", "Spotmini"]
new_encodings = tokenizer(new_titles, truncation=True, padding=True, max_length=50)

class YouTubeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])


new_dataset = YouTubeDataset(new_encodings, labels=None)
predictions = trainer.predict(new_dataset)
predicted_class_indices = predictions.predictions.argmax(axis=1)

# Convert class indices back to video IDs
predicted_video_ids = [lb.classes_[index] for index in predicted_class_indices]
# predictions
print(predicted_video_ids)

# Get the predicted titles based on the video IDs
predicted_titles = train_data[train_data['video_id'].isin(predicted_video_ids)]['title'].tolist()
print(predicted_titles)


NameError: name 'tokenizer' is not defined

- The Youtube video titled "WE WANT TO TALK ABOUT OUR MARRIAGE" can be found at https://www.youtube.com/watch?v=2kyS6SvSYSE
- the Youtube video titled "The Trump Presidency: Last Week Tonight with John Oliver (HBO)" can be found at https://www.youtube.com/watch?v=1ZAPwfrtAFY
- the Youtube video titled "Racist Superman | Rudy Mancuso, King Bach & Lele Pons" can be found at https://www.youtube.com/watch?v=5qpjK5DgCt4
- the Youtube video titles "Nickelback Lyrics: Real or Fake?" can be found at https://www.youtube.com/watch?v=puqaWrEC7tY

In [23]:
train_data[train_data['video_id']=='5qpjK5DgCt4']

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
713,5qpjK5DgCt4,17.17.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",4737269,175762,7017,9557,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
954,5qpjK5DgCt4,17.18.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",4949674,180201,7042,9687,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
238,5qpjK5DgCt4,17.15.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",4326684,167696,6730,9265,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...


In [24]:
# For prediction

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from sklearn.preprocessing import LabelBinarizer
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

import logging
logging.getLogger("transformers").setLevel(logging.ERROR)


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_path = "output/checkpoint-9500"
model = BertForSequenceClassification.from_pretrained(model_path)

new_titles = ["WE WANT TO TALK ABOUT OUR MARRIAGE", "The Trump Presidency: Last Week Tonight with John Oliver (HBO)", "Racist Superman | Rudy Mancuso, King Bach & Lele Pons", "Nickelback Lyrics: Real or Fake?"]
new_encodings = tokenizer(new_titles, truncation=True, padding=True, max_length=50)

class YouTubeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])


new_dataset = YouTubeDataset(new_encodings, labels=None)
trainer = Trainer(model=model)
predictions = trainer.predict(new_dataset)
predicted_class_indices = predictions.predictions.argmax(axis=1)

# Convert class indices back to video IDs
predicted_video_ids = [lb.classes_[index] for index in predicted_class_indices]
# predictions
print(predicted_video_ids)

# Create a dictionary to map video_id to title
video_id_to_title = dict(zip(us_videos_data['video_id'], us_videos_data['title']))

# Get the predicted titles using the dictionary
predicted_titles = [video_id_to_title[video_id] for video_id in predicted_video_ids if video_id in video_id_to_title]
print(predicted_titles)

***** Running Prediction *****
  Num examples = 4
  Batch size = 8


['2kyS6SvSYSE', '1ZAPwfrtAFY', 'nsZvmFUXkrI', 'puqaWrEC7tY']
['WE WANT TO TALK ABOUT OUR MARRIAGE', 'The Trump Presidency: Last Week Tonight with John Oliver (HBO)', 'Racist Superman | Rudy Mancuso, Alesso & King Bach', 'Nickelback Lyrics: Real or Fake?']


In [ ]:
from sklearn.metrics import precision_score

# 1. Replace new_titles with all titles from train_data
new_titles = train_data['title'].tolist()
new_encodings = tokenizer(new_titles, truncation=True, padding=True, max_length=50)

# 2. Convert tokenized data to a PyTorch dataset and predict
new_dataset = YouTubeDataset(new_encodings, labels=None)
predictions = trainer.predict(new_dataset)
predicted_class_indices = predictions.predictions.argmax(axis=1)

# Convert class indices back to video IDs
predicted_video_ids = [lb.classes_[index] for index in predicted_class_indices]

# 3. Compare these predicted IDs with the actual IDs from train_data to calculate precision
true_video_ids = train_data['video_id'].tolist()
precision_val = precision_score(true_video_ids, predicted_video_ids, average='micro')

# Results
# predicted_video_ids
precision_val

In [ ]:
# Try Using GPT2

import os
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')  # You can choose the model size: 'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'
model_path = "C:/Users/Keon/Downloads/0_2023_Thesis/LLM/nanoGPT/gpt2-medium"
model = GPT2LMHeadModel.from_pretrained(model_path)

# Use titles from train_data as input
new_titles = train_data['title'].tolist()
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
new_encodings = tokenizer(new_titles, return_tensors='pt', truncation=True, padding=True, max_length=50).input_ids

# GPT-2 doesn't use the traditional label-input format like BERT. It self-regresses to predict the next token in the sequence.
labels = new_encodings.clone()

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
new_encodings = new_encodings.to(device)
labels = labels.to(device)

# Generate predictions
with torch.no_grad():
    outputs = model.generate(new_encodings, max_length=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    predicted_video_ids = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Compare predicted video IDs with actual video IDs to calculate precision
true_video_ids = train_data['video_id'].tolist()
precision_val = precision_score(true_video_ids, predicted_video_ids, average='micro')

# Print results
print("Predicted Video IDs:", predicted_video_ids)
print("Precision:", precision_val)
